In [1]:
print("Hello world")

Hello world


In [2]:
import plotly
import plotly.express as px
import pandas as pd

In [3]:
# end = -1

end = "end"

In [4]:
# discover = 0
# profile = 0
# project = 0

discover = "discover"
profile = "profile"
project = "project"

In [5]:
# camera = 1
# photos = 1
# samples = 1
# upgrade = 1
# collection = 1

camera = "camera"
photos = "photos"
samples = "samples"
upgrade = "upgrade"
collection = "collection"

In [6]:
# usephoto = 3
usephoto = "usephoto"

In [7]:
# nextbutton = 4
# nextremove = 4
# removebackground = 4
# crop = 4

nextbutton = "nextbutton"
nextremove = "nextremove"
removebackground = "removebackground"
crop = "crop"

In [8]:
# editor = 5
editor = "editor"

In [9]:
# share = 6
# editorback = 6

share = "share"
editorback = "editorback"

In [10]:
# editorbackcancel = 7
# saveimage = 7
# saveproject = 7
# shared = 7

editorbackcancel = "editorbackcancel"
saveimage = "saveimage"
saveproject = "saveproject"
shared = "shared"

In [68]:
multi = 1000

In [69]:
t00 = [[discover, end]] * 20 * multi

t01 = [[discover, profile, discover, camera, end]] * 2 * multi
t02 = [[discover, profile, discover, photos, end]] * 2 * multi
t03 = [[discover, profile, end]] * 1 * multi

t04 = [[discover, project, discover, end]] * 70 * multi
t05 = [[discover, project, collection, project, discover, end]] * 5 * multi

t06 = [[discover, upgrade, end]] * 20 * multi

t07 = [[discover, samples, editor, end]] * 80 * multi

t08 = [[discover, camera, end]] * 400 * multi
t09 = [[discover, camera, discover, end]] * 10 * multi
t10 = [[discover, camera, discover, photos, end]] * 40 * multi

t11 = [[discover, photos, end]] * 10 * multi
t12 = [[discover, photos, usephoto, discover, end]] * 10 * multi
t13 = [[discover, photos, usephoto, nextbutton, editor, share, saveimage, end]] * 10 * multi
t14 = [[discover, photos, usephoto, nextbutton, crop, editor, end]] * 10 * multi
t15 = [[discover, photos, usephoto, nextbutton, crop, editor, editorback, saveproject, end]] * 30 * multi
t16 = [[discover, photos, usephoto, nextbutton, crop, editor, editorback, discover, end]] * 30 * multi
t17 = [[discover, photos, usephoto, removebackground, end]] * 40 * multi
t18 = [[discover, photos, usephoto, removebackground, nextremove, crop, editor, end]] * 20 * multi
t19 = [[discover, photos, usephoto, removebackground, nextremove, crop, editor, share, shared, end]] * 20 * multi
t20 = [[discover, photos, usephoto, removebackground, nextremove, editor, end]] * 20 * multi
t21 = [[discover, photos, usephoto, removebackground, nextremove, editor, editor, editor, end]] * 20 * multi
t22 = [[discover, photos, usephoto, removebackground, nextremove, editor, editor, editor, editor, end]] * 10 * multi
t23 = [[discover, photos, usephoto, removebackground, nextremove, editor, editor, editor, editor, editorback, editorbackcancel, end]] * 10 * multi
t24 = [[discover, photos, usephoto, removebackground, nextremove, editor, editor, editor, editor, share, editor, end]] * 10 * multi
t25 = [[discover, photos, usephoto, removebackground, nextremove, editor, editor, editor, editor, share, saveimage, end]] * 100 * multi

In [70]:
sessions = t00 + t01 + t02 + t03 + t04 + t05 + t06 + t07 + t08 + t09 + t10
sessions = sessions + t11 + t12 + t13 + t14 + t15 + t16 + t17 + t18 + t19 + t20
sessions = sessions + t21 + t22 + t23 + t24 + t25
len(sessions)

1000000

In [71]:
class UserBehavior:
    
    def __init__(self, sessions):
        self.sessions = sessions
        self.no_sessions = len(sessions)
        
    
    def constructor(self):
        tmp = 0
        for session in self.sessions:
            if len(session) > tmp:
                tmp = len(session)
        self.max_level = tmp - 1
        
    
    def group_by_route(self, route):
        res = []
        for session in self.sessions:
            if len(session) < len(route):
                continue
            top = session[:len(route)]
            if top == route:
                res.append(session)
        return res
    
    
    def session_occupancy(self, max_level):
        
        def next_step(route, ind):
            candidates = self.group_by_route(route)
            next_action = [candidate[ind] for candidate in candidates]
            unique_next_action = list(set(next_action))
            for action in unique_next_action:
                new_route = route + [action]
                count_action = next_action.count(action)
                if ind < max_level:
                    occupancy = count_action / self.no_sessions * 100
                    new_route_plus = new_route + [" "] * (max_level - len(new_route)) + [occupancy]
                    level[ind].append(new_route_plus)
                if action == end:
                    occupancy = count_action / self.no_sessions * 100
                else:
                    next_step(new_route, ind = ind + 1)
        
        level = dict()
        for i in range(max_level):
            level[i] = list()
        base_route = [discover]
        base_route_plus = base_route + [" "] * (max_level - 1) + [100.0]
        level[0].append(base_route_plus)
        
        next_step(base_route, ind = 1)
        
        return level

In [72]:
user_behavior = UserBehavior(sessions)
user_behavior.constructor()

In [79]:
focus_level = 12
level = user_behavior.session_occupancy(max_level = focus_level)

df_list2d = []
for ind in range(focus_level):
    for candidate in level[ind]:
        if " " in candidate:
            end_index = candidate.index(" ") - 1
            if candidate[end_index] == end:
                df_list2d.append(candidate)
        else:
            df_list2d.append(candidate)
    
df = pd.DataFrame.from_records(df_list2d)
df.columns = list(range(0, focus_level)) + ["occupancy"]
df

0        1           2                 3           4       5  \
0   discover      end                                                     
1   discover   photos         end                                         
2   discover   camera         end                                         
3   discover  upgrade         end                                         
4   discover  profile         end                                         
5   discover  samples      editor               end                       
6   discover   camera    discover               end                       
7   discover  project    discover               end                       
8   discover   photos    usephoto  removebackground         end           
9   discover   photos    usephoto          discover         end           
10  discover   camera    discover            photos         end           
11  discover  profile    discover            photos         end           
12  discover  profile    discover            camera         end           
13  discover  project  collection           project    discover     end   
14  discover   photos    usephoto  removebackground  nextremove  editor   
15  discover   photos    usephoto        nextbutton        crop  editor   
16  discover   photos    usephoto  removebackground  nextremove    crop   
17  discover   photos    usephoto        nextbutton      editor   share   
18  discover   photos    usephoto  removebackground  nextremove  editor   
19  discover   photos    usephoto        nextbutton        crop  editor   
20  discover   photos    usephoto        nextbutton        crop  editor   
21  discover   photos    usephoto  removebackground  nextremove  editor   
22  discover   photos    usephoto  removebackground  nextremove    crop   
23  discover   photos    usephoto  removebackground  nextremove  editor   
24  discover   photos    usephoto  removebackground  nextremove  editor   
25  discover   photos    usephoto  removebackground  nextremove  editor   

             6            7       8           9                10   11  \
0                                                                        
1                                                                        
2                                                                        
3                                                                        
4                                                                        
5                                                                        
6                                                                        
7                                                                        
8                                                                        
9                                                                        
10                                                                       
11                                                                       
12                                                                       
13                                                                       
14         end                                                           
15         end                                                           
16      editor          end                                              
17   saveimage          end                                              
18      editor       editor     end                                      
19  editorback  saveproject     end                                      
20  editorback     discover     end                                      
21      editor       editor  editor         end                          
22      editor        share  shared         end                          
23      editor       editor  editor  editorback  editorbackcancel  end   
24      editor       editor  editor       share         saveimage  end   
25      editor       editor  editor       share            ed

In [80]:
fig = px.sunburst(
    df, 
    path=df.columns[:-1], 
    values="occupancy", 
    width=1000, 
    height=1000,
    color=1)

## set marker colors whose labels are " " to transparent
marker_colors = list(fig.data[0].marker['colors'])
marker_labels = list(fig.data[0]['labels'])
new_marker_colors = ["rgba(0,0,0,0)" if label==" " else color for (color, label) in zip(marker_colors, marker_labels)]
marker_colors = new_marker_colors

fig.data[0].marker['colors'] = marker_colors
fig.show()

In [78]:
fig = px.sunburst(df, path=df.columns[:-1], values="occupancy", width=1000, height=1000)
fig.show()